## Imports

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

## Load MNIST dataset

In [35]:
transform = transforms.ToTensor()

In [36]:
train_data = datasets.MNIST(root='../Data', train=True, download=True, transform=transform)

In [37]:
test_data = datasets.MNIST(root='../Data', train=False, download=True, transform=transform)

In [38]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../Data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [39]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ../Data
    Split: Test
    StandardTransform
Transform: ToTensor()

## Create loaders

In [40]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [41]:
# 1 COLOR CHANNEL, 6 FILTER (OUTPUT CHANNELS), 3by3 KERNEL STRIDE
conv1 = nn.Conv2d(1,6,3,1)

# 6 Input filers (Conv1), 16 filters, 3by3 kernel, stide 1
conv2 = nn.Conv2d(6,16,3,1)

## View a sample of the train data
Note that train data will generate one single image [1,28,28] while train_loader will generate a 4D batch of 10 images [10,1,28,28] 
The 3D tensor can be viewed as a 4D tensor see below

In [55]:
for i, (X_train, y_train) in enumerate(train_data): 
    break
print(X_train.shape)

for i, (X_train_b, y_train) in enumerate(train_loader): 
    break
print(X_train_b.shape)


X_train = X_train.view(1,1,28,28)
print(X_train.shape)

torch.Size([1, 28, 28])
torch.Size([10, 1, 28, 28])
torch.Size([1, 1, 28, 28])


In [49]:
X_train.shape # Batch of 1 image

torch.Size([1, 28, 28])

## Transform to 4D (batch of 1 with 1 greyscale and 28x28)

In [46]:

X_train.view(1,1,28,28).shape

RuntimeError: shape '[1, 1, 28, 28]' is invalid for input of size 7840

In [15]:
x = X_train.view(1,1,28,28)

## Pass x into conv layer
Note that we lose information in the boarders, thus output is 26x26 instead of 28x28

In [16]:
x = F.relu(conv1(x))

In [17]:
x.shape

torch.Size([1, 6, 26, 26])

## Pass through pooling layer
note that 13x13 comes from poolin which divides the shape by 2. (28-2 = 26 / 2 = 13)

In [18]:
x = F.max_pool2d(x,2,2)

In [19]:
x.shape

torch.Size([1, 6, 13, 13])

In [20]:
x = F.relu(conv2(x))
x.shape

torch.Size([1, 16, 11, 11])

## Perform 1 additional pooling
11/2 = 5.5 which is rounded to 5

In [21]:
x = F.max_pool2d(x,2,2)

In [22]:
x.shape

torch.Size([1, 16, 5, 5])

## Flatten

In [29]:
x = x.view(-1,16*5*5)
x.shape

torch.Size([1, 400])

## Transform to output of 10

In [25]:
fc1 = nn.Linear(5*5*16,10)

In [31]:
x = fc1(x)

In [32]:
x.shape

torch.Size([1, 10])

In [33]:
x

tensor([[ 0.0801, -0.1185, -0.0846, -0.0317,  0.0441, -0.0800,  0.0888,  0.1291,
         -0.0049,  0.0564]], grad_fn=<AddmmBackward0>)